# Script to scrape Argentina Primera Division stats for the year of 2023
Author: Luis Mejia
<ol>
  <li>Imports and declarations</li>
  <li>Get the HTML code from <a href="https://es.wikipedia.org/wiki/Campeonato_de_Primera_Divisi%C3%B3n_2023_(Argentina)" target="_blank"> Wikipedia</a></li>
  <li>Scrape JSON file and load data into their own DataFrames</li>
  <li>Insert all DataFrames into my personal Google BigQuery project</li>
</ol>

In [68]:
import pandas as pd
import json
import requests
from datetime import date
from google.cloud import bigquery
import locale

client = bigquery.Client.from_service_account_json(r'service_account.json')
today = date.today()

In [69]:
# create a mapping from Spanish to English month names
month_mapping = {'enero': 'january',
                 'febrero': 'february',
                 'marzo': 'march',
                 'abril': 'april',
                 'mayo': 'may',
                 'junio': 'june',
                 'julio': 'july',
                 'agosto': 'august',
                 'septiembre': 'september',
                 'octubre': 'october',
                 'noviembre': 'november',
                 'diciembre': 'december'}

# define a function to convert the Spanish month names to English
def convert_month(fecha):
    fecha_parts = fecha.split()
    if len(fecha_parts) == 2:
        day = fecha_parts[0][:2]
        spanish_month = fecha_parts[0][2:] + " " + fecha_parts[1]
    else:
        day = fecha_parts[0]
        spanish_month = fecha_parts[2].lower()
    english_month = month_mapping.get(spanish_month.lower())
    if english_month is None:
        return fecha
    return f"{day} {english_month} 2023"

In [70]:
def import_to_gbq(df, destination_table):
    try:
        ## If you want the column and row count then
        table_id = destination_table
        job_config = bigquery.LoadJobConfig(
            schema=[
            ],
            write_disposition="WRITE_TRUNCATE",
        )

        job = client.load_table_from_dataframe(
            df, table_id, job_config=job_config
        )  # Make an API request.

        job.result()  # Wait for the job to complete.
        table = client.get_table(table_id)  # Make an API request.
        print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id))
    except Exception as e:
        print(e)

In [71]:
url = 'https://es.wikipedia.org/wiki/Campeonato_de_Primera_Divisi%C3%B3n_2023_(Argentina)'
response = requests.request("GET", url)
df = pd.read_html(response.text.encode('utf-8'))

In [72]:
#Loop through all the game dates tables and get the scores/schedule for each, and lastly append all of them into df_resultados_new dataframe
try:
    resultados_list = []
    fechas = range(11,38)
    for fecha in fechas:
        resultados = pd.DataFrame(df[fecha].values, columns = df[fecha].columns.get_level_values(1))    
        resultados_list.append(resultados)
    
    df_resultados_new = pd.concat(resultados_list)
    df_resultados_new = df_resultados_new.rename(
        columns={
            'Local': 'equipo_local', 
            'Resultado': 'resultado', 
            'Visitante': 'equipo_visitante',
            'Capacidad': 'capacidad',
            'Estadio': 'estadio',
            'Fecha': 'fecha',
            'Hora': 'hora'
            })
    
    # df_resultados_new['fecha'] = df_resultados_new['fecha'].apply(convert_month)
    df_resultados_new = df_resultados_new.assign(pais='Argentina', liga='Primera Division', temporada=2023, archive_date=today)

    df_resultados_new['fecha_juego'] = df_resultados_new['fecha'] + ' ' + str('2023')
    df_resultados_new['fecha_juego'] = df_resultados_new['fecha_juego'].apply(convert_month)
    df_resultados_new['fecha_juego'] = pd.to_datetime(df_resultados_new['fecha_juego'], format='%d %B %Y')

    # df_resultados_new.to_csv(r'C:\Users\lmeji\OneDrive\Github\Python\Python-Scripts\apis\api_argentina\csv\fechas.csv',index=False)
except Exception as e:
    print(e)        

In [74]:
#Get all other dataframes ready before going into BigQuery

#Tabla de posiciones
df_tabla_posiciones = df[8]
df_tabla_posiciones['Dif.'] = pd.to_numeric(df_tabla_posiciones['Dif.'].replace('−', '-', regex=True))
df_tabla_posiciones['Pos.'] = pd.to_numeric(df_tabla_posiciones['Pos.'].replace('.º', '', regex=True))
df_tabla_posiciones = df_tabla_posiciones.assign(pais='Argentina', liga='Primera Division', temporada=2023, archive_date=today)

df_tabla_posiciones = df_tabla_posiciones.rename(
    columns={
        'Pos.': 'posicion', 
        'Equipo': 'equipo', 
        'Pts.': 'puntos',
        'PJ': 'partidos_jugados',
        'PG': 'partidos_ganados',
        'PE': 'partidos_empatados',
        'PP': 'partidos_perdidos',
        'GF': 'goles_favor',
        'GC': 'goles_contra',
        'Dif.': 'diferencia',
        'Pais': 'pais',
        'Liga': 'liga',
        'Temporada': 'temporada',
        'Archive_date': 'archive_date',
        })

#Equipos Participantes
df_equipos_participantes = df[6]
df_equipos_participantes = df_equipos_participantes.assign(pais='Argentina', liga='Primera Division', temporada=2023, archive_date=today)
df_equipos_participantes = df_equipos_participantes.rename(
    columns={
        'Equipo.': 'equipo', 
        'Ciudad': 'ciudad', 
        'Estadio.': 'estadio',
        'Capacidad': 'capacidad',
        'Pais': 'pais',
        'Liga': 'liga',
        'Temporada': 'temporada',
        'Archive_date': 'archive_date'
        })

In [75]:
import_to_gbq(df_tabla_posiciones, 'fleet-parser-330316.luistest.futbol_tabla_posiciones')
import_to_gbq(df_equipos_participantes, 'fleet-parser-330316.luistest.futbol_equipos_participantes')
import_to_gbq(df_resultados_new, 'fleet-parser-330316.luistest.futbol_resultados')

Loaded 28 rows and 14 columns to fleet-parser-330316.luistest.futbol_tabla_posiciones
Loaded 28 rows and 8 columns to fleet-parser-330316.luistest.futbol_equipos_participantes
Loaded 378 rows and 11 columns to fleet-parser-330316.luistest.futbol_resultados
